In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import gabor
import multiprocessing

diretorio_imagens = './data/patches/'
diretorio_caracteristicas = './data/caracteristicas/'
arquivo_csv = './data/caracteristicas/Gabor.csv'

# Se o diretório não existir, cria
if not os.path.exists(diretorio_caracteristicas):
    os.makedirs(diretorio_caracteristicas)

def calcular_Gabor(imagem):
    # Parâmetros do filtro Gabor
    freqs = [0.1, 0.5, 1.0]  # Frequências do filtro
    thetas = [0, 45, 90, 135]  # Ângulos do filtro

    caracteristicas_gabor = []

    for freq in freqs:
        for theta in thetas:
            filtro_real, filtro_imag = gabor(imagem, frequency=freq, theta=theta)
            energia = np.sum(filtro_real**2 + filtro_imag**2)
            caracteristicas_gabor.append(energia)

    return caracteristicas_gabor

# Função para processar uma única imagem
def processar_imagem(image_path):
    imagem = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    caracteristicas_gabor = calcular_Gabor(imagem)
    print(f'Processando {image_path}')
    return caracteristicas_gabor

if __name__ == '__main__':
    # Inicialize um DataFrame vazio para armazenar as características Gabor
    df = pd.DataFrame()

    # Lista para armazenar resultados paralelos
    results = []

    # Crie um pool de processos para processar imagens em paralelo
    pool = multiprocessing.Pool()

    # Percorra todos os diretórios [key]
    for key in os.listdir(diretorio_imagens):
        key_path = os.path.join(diretorio_imagens, key)
        
        # Percorra todas as imagens no diretório [key]
        for image_name in os.listdir(key_path):
            image_path = os.path.join(key_path, image_name)

            # Use o pool para processar imagens em paralelo
            result = pool.apply_async(processar_imagem, (image_path,))
            results.append((key, image_name, result))

    # Aguarde a conclusão de todos os processos
    pool.close()
    pool.join()

    # Adicione as características e o nome do arquivo ao DataFrame
    for key, image_name, result in results:
        caracteristicas_gabor = result.get()
        nome_arquivo_original = image_name.split('_')[0]
        df = df.append(pd.Series([key, nome_arquivo_original] + caracteristicas_gabor), ignore_index=True)

    # Defina os nomes das colunas no DataFrame
    colunas = ['key', 'nome_arquivo'] + [f'Gabor_{i}' for i in range(len(caracteristicas_gabor))]
    df.columns = colunas

    # Salve o DataFrame em um arquivo CSV
    df.to_csv(arquivo_csv, index=False)
